In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras, os, glob, cv2
from keras.utils import np_utils

In [2]:
benign = 'benign'
malignant= 'malignant'
normal= 'normal'

ben_cases = glob.glob(benign+ '/*')
mal_cases =glob.glob(malignant+ '/*')
normal_cases = glob.glob(normal+ '/*')

#create arrays of image paths
train_mask = []
train_img=[]
y=[]

for img in normal_cases:
    if img.endswith('_mask.png') or img.endswith('_mask_1.png') or img.endswith('_mask_2.png'):  
        train_mask.append(img)
        y.append(0) #0=normal
    else:
        train_img.append(img) 

for img in ben_cases:
    if img.endswith('_mask.png') or img.endswith('_mask_1.png') or img.endswith('_mask_2.png'):
        train_mask.append(img)
        y.append(1)
    else:
        train_img.append(img)

for img in mal_cases:
    if img.endswith('_mask.png') or img.endswith('_mask_1.png') or img.endswith('_mask_2.png'):
        train_mask.append(img)
        y.append(2)
    else:
        train_img.append(img)
        
df=pd.DataFrame({'mask': train_mask, 'type': y})
print(df)

                                  mask  type
0           normal\normal (1)_mask.png     0
1          normal\normal (10)_mask.png     0
2         normal\normal (100)_mask.png     0
3         normal\normal (101)_mask.png     0
4         normal\normal (102)_mask.png     0
..                                 ...   ...
793  malignant\malignant (95)_mask.png     2
794  malignant\malignant (96)_mask.png     2
795  malignant\malignant (97)_mask.png     2
796  malignant\malignant (98)_mask.png     2
797  malignant\malignant (99)_mask.png     2

[798 rows x 2 columns]


In [3]:
import sklearn
df_rand=sklearn.utils.shuffle(df, random_state=1)
print(df_rand)

                                  mask  type
8         normal\normal (106)_mask.png     0
509       benign\benign (428)_mask.png     1
175       benign\benign (136)_mask.png     1
366       benign\benign (302)_mask.png     1
242       benign\benign (194)_mask.png     1
..                                 ...   ...
715  malignant\malignant (25)_mask.png     2
767  malignant\malignant (71)_mask.png     2
72         normal\normal (44)_mask.png     0
235       benign\benign (188)_mask.png     1
37        normal\normal (132)_mask.png     0

[798 rows x 2 columns]


In [4]:
from sklearn.model_selection import train_test_split
X=df_rand['mask']
y=df_rand['type']
#X_train,y_train,X_test,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

print(len(X))

798


In [5]:
arr = np.array([1, 2, 3, 4, 5])
arr1 = arr[:-2]
arr3 = arr[-2:]
print(len(arr))
print(arr1)
print(arr3)

5
[1 2 3]
[4 5]


In [6]:
X_train=X[:-240]
X_test=X[-240:]
y_train=y[:-240]
y_test=y[-240:]
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

558
240
558
240


In [7]:
print(np.shape(X_train))

(558,)


In [8]:
X_train_masks=[]
X_test_masks=[]
size_x = 256
size_y = 256

for every_img_path in X_train:
    img=cv2.imread(every_img_path,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(size_y, size_x))
    X_train_masks.append(img)
    
for every_img_path in X_test:
    img=cv2.imread(every_img_path,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(size_y, size_x))
    X_test_masks.append(img)
    
X_train_masks=np.array(X_train_masks)
X_test_masks=np.array(X_test_masks)

print(np.shape(X_train_masks))
print(np.shape(X_test_masks))

(558, 256, 256)
(240, 256, 256)


In [9]:
X_train_masks= X_train_masks.reshape(X_train_masks.shape[0], 
                          X_train_masks.shape[1]* X_train_masks.shape[2])
X_test_masks = X_test_masks.reshape(X_test_masks.shape[0], 
                          X_test_masks.shape[1]* X_test_masks.shape[2])

print(X_train_masks.shape[1])
X_train_masks=X_train_masks/255
X_test_masks=X_test_masks/255

65536


In [32]:
#neural net
from keras.layers import Activation, Dense
from keras.models import Sequential 
model = Sequential()
#input
model.add(Dense(200, input_dim=65536))
model.add(Activation('relu'))
#hidden layers
#num_hl = 50
#for i in range(num_hl):
model.add(Dense(50, input_dim=200))
model.add(Activation('relu'))
#output
model.add(Dense(3, input_dim=50))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 200)               13107400  
                                                                 
 activation_55 (Activation)  (None, 200)               0         
                                                                 
 dense_56 (Dense)            (None, 50)                10050     
                                                                 
 activation_56 (Activation)  (None, 50)                0         
                                                                 
 dense_57 (Dense)            (None, 3)                 153       
                                                                 
 activation_57 (Activation)  (None, 3)                 0         
                                                                 
Total params: 13,117,603
Trainable params: 13,117,603


In [33]:
model.compile(optimizer='sgd', loss='mse',metrics=['accuracy'])

model.fit(X_train_masks, y_train, batch_size=100, epochs=50, verbose=1, validation_data=(X_test_masks,y_test))

Epoch 1/50
6/6 [==============================] - 1s 88ms/step - loss: 1.0082 - accuracy: 0.6129 - val_loss: 1.0139 - val_accuracy: 0.6000
Epoch 2/50
6/6 [==============================] - 0s 62ms/step - loss: 1.0058 - accuracy: 0.5914 - val_loss: 1.0132 - val_accuracy: 0.5542
Epoch 3/50
6/6 [==============================] - 0s 75ms/step - loss: 1.0053 - accuracy: 0.5753 - val_loss: 1.0130 - val_accuracy: 0.5417
Epoch 4/50
6/6 [==============================] - 0s 63ms/step - loss: 1.0051 - accuracy: 0.5520 - val_loss: 1.0129 - val_accuracy: 0.5417
Epoch 5/50
6/6 [==============================] - 0s 63ms/step - loss: 1.0049 - accuracy: 0.5609 - val_loss: 1.0127 - val_accuracy: 0.5333
Epoch 6/50
6/6 [==============================] - 0s 70ms/step - loss: 1.0048 - accuracy: 0.5484 - val_loss: 1.0127 - val_accuracy: 0.5083
Epoch 7/50
6/6 [==============================] - 0s 62ms/step - loss: 1.0047 - accuracy: 0.5323 - val_loss: 1.0126 - val_accuracy: 0.5083
Epoch 8/50
6/6 [===========

In [34]:
#performance scores and confusion matrix
performance = model.evaluate(X_test_masks, y_test, verbose=0)

print("test loss", performance[0]) 
print("test accuracy", performance[1])

test loss 1.0120724439620972
test accuracy 0.4625000059604645
